In [12]:
# Data Wrangling
import pandas as pd
import numpy as np

# Eventual Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Text Processing
import emoji
import re
import string

# NLP
import spacy

# Spell Checking
import enchant
from autocorrect import Speller
from enchant.checker import SpellChecker

In [2]:
# Ready scraped data
df = pd.read_csv("./data/merged_output.csv")

In [29]:
# Delete reviews written in cyrillic with a mix of polish characters (Those cases are not handled by cyryllic fix later on)
df = df[~(df["offer_ref"].isin([119352308, 126556830, 97049775,]) 
          & df["entry_id"].isin([16919388, 16453304, 16368349,]))]

,entry_date,entry_id,full_category,offer_ref,product_title,purchase_date,review_text,score,top_category,sentiment
0,2021-01-17 21:55:15,13788058,Sport i rekreacja/Sporty zimowe/Sanki i ślizga...,83158301,Springos Drewniane Z Oparciem SAN001,2021-01-11 13:35:10,😑😑,1.0,Sanki,Negative
1,2021-01-17 21:55:15,15442912,Sport i rekreacja/Sporty zimowe/Sanki i ślizga...,83158301,Springos Drewniane Z Oparciem SAN001,2021-01-11 13:35:10,Lekkie i jak jesteś na kuligu to szypko spadas...,1.5,Sanki,Negative
2,2023-12-06 21:25:40,18181118,Sport i rekreacja/Sporty zimowe/Sanki i ślizga...,123702927,Ślizgacz sanki dmuchane na śnieg koło opona,2023-11-30 21:30:26,"Niestety, zamiast opony otrzymałam pingwina,wi...",1.5,Ślizgacze,Negative
3,2019-12-14 16:24:28,11430370,Sport i rekreacja/Sporty zimowe/Sanki i ślizga...,83158301,Springos Drewniane Z Oparciem SAN001,2019-12-03 19:41:18,Mogły być estetyczniej wykonane.,3.5,Sanki,Neutral
4,2023-12-15 09:33:06,18213380,Sport i rekreacja/Sporty zimowe/Sanki i ślizga...,123702927,Ślizgacz sanki dmuchane na śnieg koło opona,2023-12-11 14:15:39,Bardzo dobry,5.0,Ślizgacze,Positive
...,...,...,...,...,...,...,...,...,...,...
178786,2014-06-24 12:33:23,3083673,Uroda/Perfumy i wody/Zapachy damskie/Perfumy i...,39052947,Calvin Klein Euphoria Woda Perfumowana 100 ml,2014-06-09 18:17:17,produkt orginalny,3.5,Perfumy i wody damskie,Neutral
178787,2014-06-24 12:33:23,3118716,Uroda/Perfumy i wody/Zapachy damskie/Perfumy i...,39052947,Calvin Klein Euphoria Woda Perfumowana 100 ml,2014-06-09 18:17:17,Nie wiem czy to wina reformacji zapachu czy mo...,3.5,Perfumy i wody damskie,Neutral
179362,2013-06-25 14:02:34,10510760,Uroda/Perfumy i wody/Zapachy damskie/Perfumy i...,25152710,Hugo Boss Nuit Pour Femme Woda Perfumowana 75 ml,2014-02-24 13:26:16,Polecam ten zapach gdyż jest lekki do pomieszc...,3.5,Perfumy i wody damskie,Neutral
180034,2021-04-04 13:47:49,14630530,Uroda/Perfumy i wody/Zapachy męskie/Perfumy i ...,39817736,Christian Dior Sauvage Woda Toaletowa 100 ml,2021-03-17 13:02:54,Zapach jest nietrwały. Zamawiałam wcześniej 10...,1.5,Perfumy i wody męskie,Negative


In [3]:
# "review_text" nan are to be deleted. Other can stay.
df = df.dropna(subset=["review_text"])

# Remove invalid entries
df = df.loc[df["entry_date"] != "entry_date"]

# Remove duplicate entries
df = df.drop_duplicates(["offer_ref", "entry_id", "review_text"]).drop_duplicates("review_text")

# fix data types
df["entry_date"] = pd.to_datetime(df["entry_date"])
df["purchase_date"] = pd.to_datetime(df["purchase_date"])
df["entry_id"] = df["entry_id"].astype(int)
df["offer_ref"] = df["offer_ref"].astype(int)
df["score"] = df["score"].astype(float)

# Get Sentiment Cases based on score
df["sentiment"] = df["score"].apply(lambda x: "Positive" if x >= 4 else "Negative" if x <= 2 else "Neutral")

In [4]:
# Get only text and sentiment
to_clean = df[["review_text", "sentiment"]].copy()

In [5]:
# Get rid of newlines and other whitespace chars
to_clean["review_text"] = to_clean["review_text"].replace("[\t\r\n\v\f\ufeff]", " ", regex=True)

In [6]:
# Remove whitespaces created by previous step
to_clean["review_text"] = to_clean["review_text"].replace(" +", " ", regex=True)
to_clean[to_clean["review_text"].str.contains("  ")]

,review_text,sentiment


In [19]:
# Remove cyrillic characters. Sometimes there are reviews half written in cyrillic and half in polish, so I will keep polish characters.
cyrylic_regex = f"[ {string.punctuation}0-9]*[\u0400-\u04FF]+[ {string.punctuation}0-9]*[\u0400-\u04FF]+[ {string.punctuation}0-9]*"
to_clean["review_text"] = to_clean["review_text"].transform(lambda x: re.sub(cyrylic_regex, "", x)).replace("", np.nan)
to_clean = to_clean.dropna(subset=["review_text"])

In [7]:
# Mark all emojis in data. All_emoji will be excluded entirely, and has_emoji will remove only emojis.
to_clean["has_emoji"] = to_clean["review_text"].transform(lambda x: np.any([emoji.is_emoji(c) for c in x]))
to_clean["all_emoji"] = to_clean["review_text"].transform(lambda x: np.all([emoji.is_emoji(c) for c in x]))

In [8]:
# 460 Reviews containing emojis
print(len(to_clean[to_clean["has_emoji"]]))
# 42 reviews containing only emojis
print(len(to_clean[to_clean["all_emoji"]]))

676
55


In [9]:
# You can sort of guess the sentiment of the review based on the emojis. But it's not consistent.
to_clean[to_clean["all_emoji"]].head(10)

,review_text,sentiment,has_emoji,all_emoji
0,😑😑,Negative,True,True
1091,😐,Negative,True,True
9421,👍,Neutral,True,True
10053,👍👍👍👍👍,Positive,True,True
10958,🤗,Positive,True,True
11179,🥳🎉,Positive,True,True
13728,👍🏻👍🏻,Negative,True,True
14711,👍👍,Neutral,True,True
24204,🙂,Positive,True,True
25324,😫,Negative,True,True


In [10]:
# 1927 reviews containing special unicode characters
to_clean[to_clean["review_text"].str.contains("[\t\r\n\v\f\ufeff]")]

,review_text,sentiment,has_emoji,all_emoji


In [11]:
# Mark Ok's in data.
to_clean["has_ok"] = to_clean["review_text"].str.lower().str.contains("\Wok\W")
to_clean[to_clean["has_ok"]] = to_clean[to_clean["has_ok"]].replace(r"\bok\b", "Ok", regex=True)

In [12]:
to_clean[to_clean["has_ok"] & (to_clean["sentiment"] != "Positive")]["review_text"].iloc[2]

'Kupiłem za namową sąsiadów, którzy zachwycali się możliwością pieczenia 2 potraw jednocześnie. Dual Cooki nie są drogie, miałem Ok. 2000 zł na piekarnik (poprzedni zepsuł się po 13 latach, ale wiem, że takich urządzeń już nie ma...). Zdecydowałem się na ten model choć już w sklepie pojawiły się pierwsze wątpliwości... drzwiczki piekarnika trzaskają jak w maluchu, obudowa drzwi jest w pełni plastikowa i nie znalazłem żadnego uszczelnienie tej przegrody rozdzielającej piekarnik na 2 części. Sprzedawca stwierdził, że są to bzdety nie mające wpływu na jakość pieczenia i szczerze mówiąc przekonał mnie. Piekę głównie mięsiwa, a żona ciasta - piekarnik działa praktycznie codziennie, używamy go także do podgrzewania (nie lubię mikrofali). Rozczarowanie przyszło już po pierwszym pieczeniu - piekarnik bardzo długo się nagrzewa - temperaturę 220 stopni osiągnął po 20 minutach, mimo, że termostat pokazywał taką temperaturę już po 8-10 minutach - własny termometr pokazał, że piekarnik przekłamuje.

In [13]:
# fix_ok_in_string assumes that ok. (circa) happens once per review. We can safely assume that.
to_clean[to_clean["review_text"].transform(lambda x: len(re.findall(r"\bok\b.? [0-9]+", x))) > 1]

,review_text,sentiment,has_emoji,all_emoji,has_ok


In [14]:
# Function discriminates between ok meaning good and ok meaning circa 
# which is a short form of około in polish language.
# Ok has it's own fix because it is very common in reviews and gives positive sentiment.
def fix_ok_in_string(text):
    circa_match = re.search(r"\bok\b.? [0-9]+", text.lower())
    if circa_match is not None:
        circa_match = circa_match.span()
        return re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text[:circa_match[0]]) + text[circa_match[0]:circa_match[1]] + re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text[circa_match[1]:])
    else:
        return re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text)

In [15]:
fix_ok_in_string("ok. lalala ok. 100kg ok!")

'Ok. lalala ok. 100kg Ok!'

In [16]:
# Fix all ok's in data.
to_clean["review_text"] = to_clean["review_text"].transform(fix_ok_in_string)

In [17]:
# Remove all numbers from data.
to_clean["review_text"] = to_clean["review_text"].transform(lambda x: re.sub("[0-9]+", "", x))

In [15]:
# Largest default polish language model
nlp = spacy.load("pl_core_news_lg")

# Autocorrect speller
spell = Speller("pl", only_replacements=True)

# Enchant spellcheckers
chkr = SpellChecker("pl_Pl") 
d_typo = enchant.Dict("pl_PL")

In [16]:
to_clean["review_text"] = to_clean["review_text"].transform(lambda x: nlp(x))

In [62]:
to_clean["Typo Data"] = (
    to_clean["review_text"].transform(lambda x: 
        [[token.text, 
          d_typo.suggest(token.text), 
          i, 
          str(x).find(token.text), 
          len(token.text)] 
        for i, token in enumerate(x) 
            if not token.is_punct 
               and (not d_typo.check(token.text)) 
               and (emoji.is_emoji(token.text) == False)])
)

In [64]:
to_clean.to_pickle("./data/typos_annotated.pkl")

In [ ]:
# Will run 3 mins

# Get subset of data.
checker_test = to_clean["review_text"].iloc[:2000].transform(lambda x: nlp(x))

# 42% of first 2000 reviews contain typos
# pyenchany seems good at detecting typos however it seems bad at fixing typos So we'll have to be careful with it.
checker_test[checker_test.transform(lambda x: all([d_typo.check(token.text.lower()) for token in x if not token.is_punct])) == False]

In [20]:
# 5 Mins. tokenize data
to_clean["review_text"] = to_clean["review_text"].transform(nlp)

In [21]:
## Takes 44 mins.
## Fix all typos. Write 2 csv files with and without type fixing. Used for comparison in another script.
to_clean["review_text"].transform(lambda x: [d_typo.suggest(token.text)[0] if (not token.is_punct) and (not d_typo.check(token.text)) and (chkr.suggest(token.text))  else token for token in x]).to_csv("Testing Typo Checking.csv")
to_clean["review_text"].to_csv("Comparison to a Typo fix.csv")

In [22]:
# I've forgotten about sentiment.
to_clean["sentiment"].to_csv("sentiment provision.csv")